# Introduction
This notebook builds some models based on the features computed from the `03_feature_engineering` notebook.

# Libraries

In [3]:
import pyspark.sql.functions as f

from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.feature import VectorAssembler, StandardScaler, Normalizer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import numpy as np

# Data

Load the data and labels containing the summary features for training:

In [6]:
train_df = spark.read.parquet('dbfs:/msh/grab/data/features_valid_summary.parquet')
label_df = spark.read.parquet('dbfs:/msh/grab/data/label.parquet')

train_df = train_df.join(label_df, ['bookingID'], 'left')

display(train_df)

bookingID,trip_length,mean_Speed,sd_Speed,min_Speed,max_Speed,mean_mag_acc,sd_mag_acc,min_mag_acc,max_mag_acc,mean_mag_gyro,sd_mag_gyro,min_mag_gyro,max_mag_gyro,Speed_quantiles_0,mag_acc_quantiles_0,mag_gyro_quantiles_0,Speed_quantiles_1,mag_acc_quantiles_1,mag_gyro_quantiles_1,Speed_quantiles_2,mag_acc_quantiles_2,mag_gyro_quantiles_2,Speed_quantiles_3,mag_acc_quantiles_3,mag_gyro_quantiles_3,Speed_quantiles_4,mag_acc_quantiles_4,mag_gyro_quantiles_4,label
1030792151136,1603.0,14.862942765527583,6.797953591405691,0.32110122,25.214682,9.943056988416775,0.9843915474530084,4.308710817491772,16.859894585414715,0.08057330483258394,0.06612632110588129,3.3848991871423775E-4,0.5972424523416325,4.36959,8.468746595984339,0.012190015840400302,8.907368,9.401708510133178,0.03605472949018259,15.346053,9.949168134488577,0.0652902712283898,21.219063,10.456675929801337,0.103559177781611,24.515537,11.559472330712712,0.1968506852315648,0
1125281431735,2540.0,8.752515809461885,5.337300086897193,0.27703157,23.360136,9.8770982695034,0.48576986790228505,7.8596431414000945,15.022379838624422,0.05151536792035864,0.08689249419154112,7.65180618865624E-4,0.7376963727607299,1.0653919,9.154616093045592,0.004094121349716729,4.4294543,9.646897500689462,0.01372298960187753,8.478179,9.852513436892071,0.02709716654029122,12.170646,10.087294689812198,0.04896767998337855,18.379423,10.679506025472351,0.20407437708117657,1
1133871366236,1304.0,19.154742669274608,5.859948872165777,0.55805635,26.053034,9.904921671463885,0.7990869543290869,6.3461444874767095,14.37671663032197,0.1393655267595005,0.11019204122600985,0.001919825127757817,0.8984207625021929,6.150825,8.756190952259916,0.026449456825521436,17.531254,9.46101156073323,0.06730467419748215,21.343855,9.87434780919104,0.11217206458172845,23.173452,10.286021735286079,0.1756743768288079,24.46182,11.235556791133574,0.33564584274863185,0
1159641170054,1439.0,14.764780104791978,9.376679853244744,0.050017327070236206,28.254985809326172,9.906467882086517,0.45000038005997384,7.776839038612129,12.127667124903473,0.07712339104069614,0.07336093215581906,0.0010831082046748694,0.7734178114099859,0.22828109562397003,9.18016321038499,0.010755176227145787,5.591987609863281,9.709562729605953,0.027184978194705393,15.424273490905762,9.866645902071086,0.05919470494870196,23.98752212524414,10.137274036744145,0.10283203389879254,26.23044776916504,10.671712860751827,0.19979074060125696,1
1211180777517,1679.0,4.771441405954892,4.376222979554796,0.027965625748038292,16.977375030517578,9.790380067507114,0.5854900593925125,5.026376665818665,13.466223628091,0.07759626947740124,0.07888580996359348,0.001516055133951715,0.5641418281465412,0.1645251363515854,8.926855339260472,0.02367609809366189,2.2365331649780273,9.550132858188418,0.03791656551021414,2.7477641105651855,9.767924858635139,0.04765099924186613,7.738240718841553,10.034411755478438,0.08593814051591614,13.856679916381836,10.713573795493827,0.24137231722642846,1
1228360646715,1289.0,10.388828020078712,6.911426778578952,0.036837972700595856,23.786556243896484,9.945654867895508,0.6221742634734229,7.174647985045568,15.524251862610441,0.16176089106405628,0.15367267150976396,5.522433606530815E-4,1.2105769386589718,0.13670353591442108,9.008243268424026,0.009202034233831915,3.405956983566284,9.659436935669486,0.05160543171836655,10.68129825592041,9.86505693887842,0.12274074195770855,16.491731643676758,10.266540889048628,0.22313545887123826,20.82914924621582,10.974619936969372,0.4578494290428183,0
1279900254316,1528.0,3.6195460717246783,2.221900117027084,0.17000000178813934,9.390000343322754,11.198731215939466,5.540173218293301,0.45400413576564624,83.28288662996448,0.6879455235162257,0.8323649887123413,0.002082969548574677,9.71538273532383,0.23000000417232513,5.094503288015166,0.02892813104871178,2.2300000190734863,9.056834915521744,0.16363447687696142,3.069999933242798,9.863139468030436,0.4467578790714034,5.389999866485596,12.241650794194294,0.887924832467014,7.769999980926514,19.624143195

# Models

Pipeline stuff:

In [9]:
input_columns = train_df.columns[1:-1]
vec_assembler = VectorAssembler(inputCols=input_columns, outputCol='features')
scaler = StandardScaler(inputCol='features', outputCol='features_std', withStd=True, withMean=True)

lr = LogisticRegression(featuresCol='features_std')
rf = RandomForestClassifier(featuresCol='features_std')
gbt = GBTClassifier(featuresCol='features_std')

Logistic Regression:

In [11]:
%%time
pipeline = Pipeline(stages=[vec_assembler, scaler, lr])

paramGrid = ParamGridBuilder()\
  .addGrid(lr.regParam, [0.1, 0.01])\
  .addGrid(lr.maxIter, [10, 50, 100, 150])\
  .addGrid(lr.elasticNetParam, [0, 0.1, 0.01])\
  .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(metricName='areaUnderROC'),
                          numFolds=10)

cv_lr = crossval.fit(train_df)

auc_lr = max(cv_lr.avgMetrics)

print(f'Best AUC from logistic regression: {auc_lr:.4f}')

/databricks/spark/python/pyspark/ml/util.py:751: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)
Best AUC from logistic regression: 0.6765
CPU times: user 20 s, sys: 5.55 s, total: 25.5 s
Wall time: 16min 18s

Random Forest:

In [13]:
%%time
pipeline = Pipeline(stages=[vec_assembler, scaler, rf])

paramGrid = ParamGridBuilder()\
  .addGrid(rf.maxDepth, [1, 5, 10, 30])\
  .addGrid(rf.numTrees, [10, 20, 50, 100])\
  .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(metricName='areaUnderROC'),
                          numFolds=10)

cv_rf = crossval.fit(train_df)

auc_rf = max(cv_rf.avgMetrics)

print(f'Best AUC from random forest: {auc_rf:.4f}')

/databricks/spark/python/pyspark/ml/util.py:751: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)
Best AUC from random forest: 0.6707
CPU times: user 17.3 s, sys: 5.02 s, total: 22.4 s
Wall time: 21min 50s

GBT:

In [15]:
%%time
pipeline = Pipeline(stages=[vec_assembler, scaler, rf])

paramGrid = ParamGridBuilder()\
  .addGrid(gbt.maxDepth, [5, 10, 20])\
  .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(metricName='areaUnderROC'),
                          numFolds=10)

cv_gbt = crossval.fit(train_df)

auc_gbt = max(cv_gbt.avgMetrics)

print(f'Best AUC from GBT: {auc_gbt:.4f}')

/databricks/spark/python/pyspark/ml/util.py:751: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)
Best AUC from GBT: 0.6646
CPU times: user 2.62 s, sys: 816 ms, total: 3.43 s
Wall time: 2min 27s

Pick the best model:

In [17]:
aucs = [auc_gbt, auc_lr, auc_rf]
cvs = [cv_gbt, cv_lr, cv_rf]

best_idx = np.argmax(aucs)
best_cv = cvs[best_idx]

Save the best model

In [19]:
%fs
mkdirs /msh/grab/model/

res8: Boolean = true

In [20]:
best_cv.bestModel.write().overwrite().save('dbfs:/msh/grab/model/best_model')

In [21]:
%fs
ls /msh/grab/model/

path,name,size
dbfs:/msh/grab/model/best_model/,best_model/,0
